In [11]:
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy import signal
import scipy.integrate as integrate
import scipy.fftpack as ff
#!/usr/bin/env python

import numpy as np, os, sys, joblib
from scipy.io import loadmat
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from get_12ECG_features import get_12ECG_features

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path
from numpy import genfromtxt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
import keras

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Signal lecture 
#### Please note: 
For easier data analysis and features visualization, please use pandas (since its easier to use). But for the final signal (the one we need to integration), please use numpy format (convert to numpy using "signal.values")

In [13]:
features = f'features.csv'
labels = f'labels.csv'
testing = f'testing.csv'
testing = genfromtxt(testing, delimiter=',')
features = genfromtxt(features, delimiter=',')
labels = genfromtxt(labels, delimiter=',')

In [15]:
# Normalize data
testing = tf.keras.utils.normalize(testing)
features = tf.keras.utils.normalize(features)

In [17]:
print(features.dtype)
print(testing.dtype)
print(label.dtype)

float64
float64
float64


In [19]:
## Inside the run_12ECG_code we must consider the following structure (1,14)
testing_reshape = testing.reshape(1, -1)
imputer=SimpleImputer().fit(testing_reshape)
testing_reshape = imputer.transform(testing_reshape)
print(testing_reshape.shape)

(1, 14)


In [21]:
# Training set bust consider the next structure
print(features.shape, labels.shape)
4812*14

(4813, 14) (4813, 9)


67368

In [95]:
# Since labels are organized in hot-encoding format and we need the index Number, define decode:
def decode(datum):
    return np.argmax(datum)
label_final = []
    
# Extract hot-encode format to normal format
for i in range(labels.shape[0]):
    label = labels[i]
    decoded_labels = decode(label)
    label_final.append(decoded_labels)
# Convert list to np array
label_final = np.array(label_final)
label_final = np.uint8(label_final)
print(label_final.dtype)

uint8


In [97]:
label_final.shape

(4813,)

#### Model 1 : branch NN_1

In [24]:
print("training model NN_1")
############# MODEL 1 - NN ######################
model_1 = tf.keras.models.Sequential()
model_1.add(tf.keras.layers.Flatten())
# use 128 neurons & use relu act func
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
# the final output layer must be Dense and must fit # classifications and must use probability distribution instead of an activation function
model_1.add(tf.keras.layers.Dense(9, activation = tf.nn.softmax))
    
# Compile 
model_1.compile(
optimizer= 'adam' ,# NN intends to minimize losss, not maximize accuracy
loss = 'sparse_categorical_crossentropy',
metrics = ['accuracy']
)
# Fit the model appropiatelly
model_1.fit(features, label_final, epochs=10)
val_loss, val_acc = model_1.evaluate(features, label_final)
print("The model is trained")

training model NN_1
Epoch 1/10
151/151 [==============================] - 0s 2ms/step - loss: 2.0685 - accuracy: 0.2182
Epoch 2/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0555 - accuracy: 0.2256
Epoch 3/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0532 - accuracy: 0.2256
Epoch 4/10
151/151 [==============================] - 0s 2ms/step - loss: 2.0553 - accuracy: 0.2256
Epoch 5/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0544 - accuracy: 0.2256
Epoch 6/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0522 - accuracy: 0.2256
Epoch 7/10
151/151 [==============================] - 0s 2ms/step - loss: 2.0533 - accuracy: 0.2256
Epoch 8/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0524 - accuracy: 0.2256
Epoch 9/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0518 - accuracy: 0.2256
Epoch 10/10
151/151 [==============================] - 0s 739us/step - loss: 2.0

In [25]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 14)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               1920      
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_12 (Dense)             (None, 128)              

In [26]:
# Save model.
model_1.save("NN_1.model")

INFO:tensorflow:Assets written to: NN_1.model\assets


In [27]:
print(features.shape, label_final.shape)

(4813, 14) (4813,)


In [29]:
nn_model = tf.keras.models.load_model('NN_1.model/')
nn_model

### current label & score
1. From the baseline code we had that the original model saved three things:`model`, `imputer`, `classes` on a file called `finalized_model.sav` from the joblib library. 
2. The Imputer used in the training code might be affecting the testing set. Imputer replaces np.nan values inside the features and it is saved only once for the entire traininset, therefore it must be saved to be re used in the driver.py code
3. `current_label`: is originally extracted using:

 `current_label = model.predict(feats_reshape)[0]`:  Array of float64 (9,) # array([0., 1., 0., 0., 0., 0., 0., 0., 1.])
 
 `current_label=current_label.astype(int)`:  Array of int32 (9,) # array([0, 1, 0, 0, 0, 0, 0, 0, 1])

 `current_score = model.predict_proba(feats_reshape)`: List of 9 which is converted to np.array # array([0.04, 0.74, 0.02, 0.04, 0.09, 0.02, 0.  , 0.03, 0.72])

 `classes`: This function is stored as well and contains the classes tags:  
 
 classes = ['164884008', '164889003', '164909002', '164931005', '270492004', '284470004', '426783006', '429622005', '59118001']
   

In [34]:
# adding [0] for extracting the values from the array
current_label = nn_model.predict([testing_reshape])[0]
current_label = current_label.astype(int)
current_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

Evidently this will be zero all the time unless prediction is really high. Therefore we can use argmax.

In [37]:
current_label = nn_model.predict(testing_reshape)[0]
current_label

array([0.09403204, 0.16651522, 0.02546168, 0.03028586, 0.09770199,
       0.08466987, 0.1342014 , 0.11385558, 0.25327644], dtype=float32)

In [38]:
# Using the baseline probaility we obtain:
current_score = nn_model.predict_proba(testing_reshape)
current_score

Instructions for updating:
Please use `model.predict()` instead.


array([[0.09403204, 0.16651522, 0.02546168, 0.03028586, 0.09770199,
        0.08466987, 0.1342014 , 0.11385558, 0.25327644]], dtype=float32)

In [40]:
# Suppose softmax model 
probability_model = tf.keras.Sequential([nn_model,tf.keras.layers.Softmax()])  
current_score = nn_model.predict(testing_reshape)
current_score

array([[0.09403204, 0.16651522, 0.02546168, 0.03028586, 0.09770199,
        0.08466987, 0.1342014 , 0.11385558, 0.25327644]], dtype=float32)

From this results we can infer that nn_model.predict is equivalent to probability_model from Softmax performace

Conclusion

In [56]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

prediction = nn_model.predict([testing_reshape])[0]
print(prediction)
prediction = np.argmax(prediction)
encoded = to_categorical(prediction)
current_label = encoded.astype(int)
print(current_label)

[0.09403204 0.16651522 0.02546168 0.03028586 0.09770199 0.08466987
 0.1342014  0.11385558 0.25327644]
[0 0 0 0 0 0 0 0 1]


In [63]:
test = [0.9403204, 0.16651522, 0.02546168, 0.03028586, 0.09770199, 0.08466987, 0.1342014,  0.11385558, 0.9403204]

In [64]:
test = np.array(test)
test

array([0.9403204 , 0.16651522, 0.02546168, 0.03028586, 0.09770199,
       0.08466987, 0.1342014 , 0.11385558, 0.9403204 ])

In [69]:
current_label = test.astype(int)
current_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Model 2: CNN

In [98]:
sequence_size = features.shape[1]
n_features=1
print(features.shape, label_final.shape)

(4813, 14) (4813,)


In [103]:
cnn_model = Sequential([
Conv1D(
    filters=1,
    kernel_size=4,
    strides=1,
    input_shape=(14, 1),
    padding="same",
    activation="relu"
),
Flatten(),
Dropout(0.5),
Dense(
    1,
    activation="sigmoid",
    name="output",
)
])
optimizer = Adam(lr=0.001)
# Compiling the model
cnn_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 14, 1)             5         
_________________________________________________________________
flatten_7 (Flatten)          (None, 14)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 14)                0         
_________________________________________________________________
output (Dense)               (None, 1)                 15        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [104]:
feat_final = np.expand_dims(features, axis=2) 
label_final = label_final.reshape(-1,1)
print("features shape: ", feat_final.shape)
print("label features: ", label_final.shape)

features shape:  (4813, 14, 1)
label features:  (4813, 1)


In [105]:
label_final.shape

(4813, 1)

In [106]:
cnn_model.fit(feat_final, label_final)

ValueError: `updates` argument is not supported during eager execution. You passed: [<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=0>, <tf.Variable 'UnreadVariable' shape=(4, 1, 1) dtype=float32, numpy=
array([[[0.]],

       [[0.]],

       [[0.]],

       [[0.]]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(4, 1, 1) dtype=float32, numpy=
array([[[0.]],

       [[0.]],

       [[0.]],

       [[0.]]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(4, 1, 1) dtype=float32, numpy=
array([[[-0.6989074 ]],

       [[ 0.46536762]],

       [[-0.36386883]],

       [[ 0.20603174]]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(14, 1) dtype=float32, numpy=
array([[0.],
       [0.],
     